In [ ]:
import numpy as np
import xarray as xr

import time
import argparse
from xrspatial.zonal import stats
from xrspatial.utils import has_cuda
from xrspatial.utils import doesnt_have_cuda

In [ ]:
def create_arr(data=None, H=10, W=10, backend='numpy'):
    assert(backend in ['numpy', 'cupy', 'dask'])
    if data is None:
        data = np.zeros((H, W), dtype=np.float32)
    raster = xr.DataArray(data, dims=['y', 'x'])

    if has_cuda() and 'cupy' in backend:
        import cupy
        raster.data = cupy.asarray(raster.data)

    if 'dask' in backend:
        import dask.array as da
        raster.data = da.from_array(raster.data, chunks=(10, 10))

    return raster

In [ ]:
class Dummy:
    pass
args = Dummy()
args.W = 10
args.H = 10
args.zH = 2
args.zW = 2
args.iterations = 1
args.backend = 'numpy'

In [ ]:
H = args.H
W = args.W
zH = args.zH
zW = args.zW
assert(W/zW == W//zW)
assert(H/zH == H//zH)

In [ ]:
# Values raster
values = xr.DataArray(np.arange(H * W).reshape(H, W))
values = create_arr(values, backend=args.backend)
print(values)

In [ ]:
# Zones raster
hstep = H//zH
wstep = W//zW
zones = xr.DataArray(np.zeros(H * W).reshape(H, W))

In [ ]:
for i in range(zH):
    for j in range(zW):
        zones[i * hstep: (i+1)*hstep, j*wstep: (j+1)*wstep] = i*zW + j
print(zones)

In [ ]:
# Profiling region, warming up
start = time.time()
stats_df = stats(zones=zones, values=values)
warm_up_sec = time.time() - start

In [ ]:
# main calculation loop
elapsed_sec = 0
for i in range(args.iterations):
    start = time.time()
    stats_df = stats(zones=zones, values=values)
    # if args.backend == 'dask':
    #     stats_df = stats_df.compute()
    elapsed_sec += time.time() - start

In [ ]:
# Timing report
print('HxW,Runs,total_time(sec),time_per_run(sec),warm_up_time(sec)')
print('{}x{},{},{:.4f},{:.4f},{:.4f}'.format(
    H, W,
    args.iterations, elapsed_sec,
    elapsed_sec/args.iterations, warm_up_sec))

In [ ]:
# results report
print('Result: ', stats_df)